In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from IPython.display import HTML

# Load the data
df = pd.read_csv('/Users/dagmarrothschild/Downloads/FemaWebDisasterDeclarations_formatted_3.csv', index_col=0)

# Convert the index (date column) to datetime
df.index = pd.to_datetime(df.index)

# Extract year and quarter from the date index
df['Year'] = df.index.year
df['Quarter_Num'] = df.index.quarter

# Filter for 2022 Q1 onwards
df = df[df.index >= '2022-01-01']

# Create a year-quarter column for grouping
df['Year_Quarter'] = df['Year'].astype(str) + '-Q' + df['Quarter_Num'].astype(str)

# Create interactive visualization with Plotly
fig = go.Figure()

# Aggregate for quarterly statistics
quarterly_stats = df.groupby('Year_Quarter')['Days Diff'].agg(['mean', 'std', 'count']).reset_index()
quarterly_stats.columns = ['Year_Quarter', 'Avg_Days_Diff', 'Std_Days_Diff', 'Declaration_Count']

# Add quarterly average line with error bars for ALL quarters
fig.add_trace(go.Scatter(
    x=quarterly_stats['Year_Quarter'],
    y=quarterly_stats['Avg_Days_Diff'],
    mode='lines+markers',
    line=dict(color='#1a1a1a', width=4),
    marker=dict(size=12, color=['#e74c3c' if 'Q4' in yq and '2025' in yq else '#1a1a1a' for yq in quarterly_stats['Year_Quarter']], 
               line=dict(color='white', width=3)),
    error_y=dict(
        type='data',
        array=quarterly_stats['Std_Days_Diff'],
        color='#34495e',
        thickness=2,
        width=6
    ),
    hovertemplate='<b>%{x}</b><br><b>Avg: %{y:.1f}</b> days<br><b>Std Dev: ±%{error_y.array:.1f}</b><extra></extra>',
    name='Quarterly Average',
    showlegend=True
))

# Add individual data points AFTER (so they appear above)
fig.add_trace(go.Scatter(
    x=df['Year_Quarter'],
    y=df['Days Diff'],
    mode='markers',
    marker=dict(size=5, color='#2c3e50', opacity=0.5, line=dict(width=0)),
    hovertemplate='<b>Days:</b> %{y}<extra></extra>',
    name='Individual Declarations',
    showlegend=True
))

# Find peak
peak_idx = quarterly_stats['Avg_Days_Diff'].idxmax()
peak_quarter = quarterly_stats.loc[peak_idx, 'Year_Quarter']
peak_value = quarterly_stats.loc[peak_idx, 'Avg_Days_Diff']

# Add annotation for December 2023 - Pre-Disaster Mitigation grant program
fig.add_annotation(
    x='2023-Q4',
    y=-0.07,
    xref='x',
    yref='paper',
    text='<b>December 2023</b><br>FEMA introduces the<br>Pre-Disaster Mitigation<br>grant program',
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=2,
    arrowcolor='#3498db',
    ax=0,
    ay=40,
    xanchor='center',
    yanchor='top',
    bgcolor='rgba(52, 152, 219, 0.08)',
    bordercolor='#3498db',
    borderwidth=1.5,
    borderpad=10,
    font=dict(color='#2c3e50', size=9, family='Inter, -apple-system, BlinkMacSystemFont, sans-serif'),
    align='center'
)

# Add annotation for May 2025 policy change - box on left, thin arrow
fig.add_annotation(
    x='2025-Q2',
    y=-0.07,
    xref='x',
    yref='paper',
    text='<b>May 2025</b><br>FEMA staff reduced 1/3<br>$645M budget cut',
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=1,
    arrowcolor='#e74c3c',
    ax=-80,
    ay=10,
    xanchor='right',
    yanchor='top',
    bgcolor='rgba(231, 76, 60, 0.08)',
    bordercolor='#e74c3c',
    borderwidth=1.5,
    borderpad=10,
    font=dict(color='#2c3e50', size=9, family='Inter, -apple-system, BlinkMacSystemFont, sans-serif'),
    align='center'
)

# Add annotation for October 2025 policy change - positioned in bottom margin
fig.add_annotation(
    x='2025-Q4',
    y=-0.07,
    xref='x',
    yref='paper',
    text='<b>October 2025</b><br>FEMA canceled $11B<br>in payments',
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=2,
    arrowcolor='#e74c3c',
    ax=80,
    ay=40,
    xanchor='center',
    yanchor='top',
    bgcolor='rgba(231, 76, 60, 0.08)',
    bordercolor='#e74c3c',
    borderwidth=1.5,
    borderpad=10,
    font=dict(color='#2c3e50', size=9, family='Inter, -apple-system, BlinkMacSystemFont, sans-serif'),
    align='center'
)

fig.update_layout(
    title=dict(
        text='<b>FEMA Disaster Declaration Processing Reaches Record High in Q4 2025</b><br><sub style="font-weight: normal; font-size: 14px; color: #7f8c8d;">Showing impact of policy changes on processing times</sub>',
        x=0.02,
        xanchor='left',
        font=dict(size=26, color='#1a1a1a', family='-apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif'),
        y=0.97,
        yanchor='top'
    ),
    xaxis_title='',
    yaxis_title='Average Processing Time (Days)',
    hovermode='closest',
    template='plotly_white',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    font=dict(family='-apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif', size=11, color='#4a4a4a'),
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='#e0e0e0',
        zeroline=False,
        tickfont=dict(size=10, color='#757575')
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=0.5,
        gridcolor='#f5f5f5',
        showline=True,
        linewidth=1,
        linecolor='#e0e0e0',
        zeroline=False,
        range=[0, None],
        tickfont=dict(size=10, color='#757575'),
        title_font=dict(size=12, color='#1a1a1a', family='-apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif')
    ),
    width=1100,
    height=650,
    margin=dict(l=80, r=50, t=150, b=200),
    showlegend=True,
    legend=dict(
        x=0.99,
        y=0.99,
        xanchor='right',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.85)',
        bordercolor='#e0e0e0',
        borderwidth=1
    )
)

graph_html = fig.to_html(include_plotlyjs='cdn')

# Build complete page HTML
page_html = f"""
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>FEMA Processing Speed Bump</title>
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin="">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link href="https://fonts.googleapis.com/css2?family=Lora:ital,wght@0,400;0,700;1,400;1,700&amp;display=swap" rel="stylesheet">
    <style>
        body {{
            margin: 0;
            font-size: 20px;
            font-family: Georgia, 'Times New Roman', Times, serif;
            text-rendering: optimizeLegibility;
        }}
        .content {{
            max-width: 640px;
            margin: auto;
            padding: 2em 1em;
        }}
        a {{ color: #f05349; }}
        p {{
            line-height: 1.6;
            margin: 0;
            padding-bottom: 1.2em;
        }}
        .byline {{
            font-size: 0.95em;
            color: #666;
            margin-bottom: 1.5em;
        }}
        .byline p {{
            margin: 0;
            padding: 0;
        }}
        .quote {{
            font-style: italic;
            color: #555;
            margin: 1.5em 0;
            padding-left: 1.5em;
            border-left: 4px solid #A23B72;
        }}
        .quote-attribution {{
            font-size: 0.95em;
            color: #888;
            margin: 0.5em 0 1.5em 0;
            padding: 0;
        }}
        .hero {{
            position: relative;
            width: 100%;
            height: 600px;
            overflow: hidden;
            display: flex;
            align-items: center;
            justify-content: center;
        }}
        .hero video {{
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            object-fit: cover;
        }}
        .hero-content {{
            position: absolute;
            text-align: center;
            color: #fff;
            z-index: 10;
            width: 95%;
            max-width: 1000px;
            top: 35%;
            left: 50%;
            transform: translate(-50%, -50%);
        }}
        .hero h1 {{
            color: #fff;
            font-size: 4em;
            font-weight: 900;
            margin: 0;
            padding: 1.2em 1.5em;
            letter-spacing: -1px;
            line-height: 1.1;
            display: inline-block;
            background-color: rgba(0, 0, 0, 0.45);
            border-radius: 8px;
        }}
        .graph-container {{
            width: 100%;
            display: flex;
            justify-content: center;
            margin: 3em auto 2em auto;
            padding: 0;
        }}
        .graph-wrapper {{
            width: fit-content;
            max-width: 90vw;
        }}
        .footer {{
            background-color: #f5f5f5;
            padding: 2em 1em;
            margin-top: 3em;
            border-top: 1px solid #e0e0e0;
        }}
        .footer-content {{
            max-width: 640px;
            margin: 0 auto;
            font-size: 0.85em;
            color: #666;
        }}
        .footer-content p {{
            padding-bottom: 0.5em;
        }}
        @media (max-width: 640px) {{
            body {{ font-size: 18px; }}
            .content {{ padding: 1.5em 0.5em; }}
            .hero {{ height: 400px; }}
            .hero h1 {{ font-size: 2.2em; padding: 0.8em 1em; }}
        }}
    </style>
</head>
<body>
    <div class="hero">
        <video src="FEMA.mp4" preload="auto" tabindex="0" playsinline="" autoplay="" loop="" muted=""></video>
        <div class="hero-content">
            <h1>Double Time: Tracking FEMA's Processing Speed Bump</h1>
        </div>
    </div>
    <div class="content">
        <div class="byline">
            <p>By <a href="https://dagmarroth.github.io/Website/">Dagmar Rothschild</a></p>
        </div>
        <p>Following significant budget cuts and staff reductions at FEMA in 2025, disaster declaration processing times doubled in the fourth quarter. According to Shana Udvardy, Senior Analyst at the Union of Concerned Scientists' Climate and Energy Program, the reason is straightforward: fewer people processing the same volume of applications creates bottlenecks that cascade through the system. The delays are particularly concerning because they compound during the exact periods when disaster victims need help most.</p>
        <div class="quote">
            "When Hurricane Helene hit our farm in western North Carolina, we lost everything—the crops, the equipment, part of the barn. We filed for disaster assistance right away, but we've been waiting months now. The delays just keep piling up, and meanwhile we're trying to figure out how to survive the winter and plant next season."
            <p class="quote-attribution">— Justin Rhodes, western North Carolina farmer</p>
        </div>
    </div>
    <div class="graph-container">
        <div class="graph-wrapper">
            {graph_html}
        </div>
    </div>
    <div class="footer">
        <div class="footer-content">
            <p><strong>Data Source:</strong> <a href="https://www.fema.gov/openfema-data-page/fema-web-disaster-declarations-v1">FEMA Web Disaster Declarations (v1)</a></p>
        </div>
    </div>
</body>
</html>
"""

HTML(page_html)